In [ ]:
from __future__ import print_function
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import os
from datetime import datetime
import io
import itertools
from packaging import version
from six.moves import range
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn.metrics

batch_size = 256
num_classes = 50
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar50_trained_model.h5'

class_names = np.array([
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 
    'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 
    'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 
    'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 
    'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 
    'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion',
    'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse',
    'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear',
    'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine',
    'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose',
    'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake',
    'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table',
    'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout',
    'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman',
    'worm'
])

class_names = class_names[1::2]

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
train_filter = y_train % 2 != 0
test_filter = y_test % 2 != 0
train_filter = train_filter.flatten()
test_filter = test_filter.flatten()
x_train = x_train[train_filter]
y_train = y_train[train_filter]
x_test = x_test[test_filter]
y_test = y_test[test_filter]
y_train = np.array(list(map(lambda x: (x - 1) / 2, y_train)))
y_test = np.array(list(map(lambda x: (x - 1) / 2, y_test)))

logdir = "logs/plots/" + datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

def log_confusion_matrix(epoch, logs):
  # Use the model to predict the values from the validation dataset.
  test_pred_raw = model.predict(x_test)
  test_pred = np.argmax(test_pred_raw, axis=1)

  # Calculate the confusion matrix.
  cm = sklearn.metrics.confusion_matrix(y_test, test_pred)
  # Log the confusion matrix as an image summary.
  figure = plot_confusion_matrix(cm, class_names=class_names)
  cm_image = plot_to_image(figure)

  # Log the confusion matrix as an image summary.
  with file_writer_cm.as_default():
    tf.summary.image("Confusion Matrix", cm_image, step=epoch)

# Define the per-epoch callback.
cm_callback = keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

def plot_confusion_matrix(cm, class_names):
  """
  Returns a matplotlib figure containing the plotted confusion matrix.

  Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
  """
  figure = plt.figure(figsize=(40, 40))
  plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
  plt.title("Confusion matrix")
  plt.colorbar()
  tick_marks = np.arange(len(class_names))
  plt.xticks(tick_marks, class_names, rotation=45)
  plt.yticks(tick_marks, class_names)

  # Normalize the confusion matrix.
  cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

  # Use white text if squares are dark; otherwise black.
  threshold = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    color = "white" if cm[i, j] > threshold else "black"
    plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  return figure

def plot_to_image(figure):
  """Converts the matplotlib plot specified by 'figure' to a PNG image and
  returns it. The supplied figure is closed and inaccessible after this call."""
  # Save the plot to a PNG in memory.
  buf = io.BytesIO()
  plt.savefig(buf, format='png')
  # Closing the figure prevents it from being displayed directly inside
  # the notebook.
  plt.close(figure)
  buf.seek(0)
  # Convert PNG buffer to TF image
  image = tf.image.decode_png(buf.getvalue(), channels=4)
  # Add the batch dimension
  image = tf.expand_dims(image, 0)
  return image

def image_grid():
  """Return a 5x5 grid of the MNIST images as a matplotlib figure."""
  # Create a figure to contain the plot.
  figure = plt.figure(figsize=(10,10))
  for i in range(25):
    # Start next subplot.
    plt.subplot(5, 5, i + 1, title=class_names[int(y_train[i][0])])
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i], cmap=plt.cm.binary)
  
  return figure

# Prepare the plot
figure = image_grid()
# Convert to image and log
with file_writer.as_default():
  tf.summary.image("Training data", plot_to_image(figure), step=0)
# Convert class vectors to binary class matrices.
y_train_matrix = keras.utils.to_categorical(y_train, num_classes)
y_test_matrix = keras.utils.to_categorical(y_test, num_classes)

if os.path.exists(os.path.join(save_dir, model_name)):
  model = keras.models.load_model(os.path.join(save_dir, model_name))
  print("model loaded")
else:
  model = Sequential()

  model.add(Conv2D(128, (3, 3), padding='same', input_shape=x_train.shape[1:]))
  model.add(Activation('elu'))
  model.add(Conv2D(128, (3, 3)))
  model.add(Activation('elu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.1))

  model.add(Conv2D(256, (3, 3), padding='same'))
  model.add(Activation('elu'))
  model.add(Conv2D(256, (3, 3)))
  model.add(Activation('elu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(512, (3, 3), padding='same'))
  model.add(Activation('elu'))
  model.add(Conv2D(512, (3, 3)))
  model.add(Activation('elu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.5))

  model.add(Flatten())
  model.add(Dense(1024))
  model.add(Activation('elu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))

  # initiate RMSprop optimizer
  opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

  # Let's train the model using RMSprop
  model.compile(loss='categorical_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])
  print("new model created")

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

logdir = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)


# Sets up a timestamped log directory.
logdir = "logs/train_data/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

with file_writer.as_default():
  # Don't forget to reshape.
  images = np.reshape(x_train[0:25], (-1, 32, 32, 3))
  tf.summary.image("25 training data examples", images, max_outputs=25, step=0)



if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train_matrix,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test_matrix),
              shuffle=True,
              callbacks=[tensorboard_callback, cm_callback])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit(datagen.flow(x_train, y_train_matrix,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test_matrix),
                        callbacks=[tensorboard_callback, cm_callback])

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test_matrix, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

!zip -r /content/file.zip /content
from google.colab import files
files.download("/content/file.zip")

model loaded
Using real-time data augmentation.
Epoch 1/100
98/98 [==============================] - 24s 246ms/step - loss: 0.1820 - accuracy: 0.9413 - val_loss: 1.2248 - val_accuracy: 0.7518
Epoch 2/100
98/98 [==============================] - 22s 224ms/step - loss: 0.1854 - accuracy: 0.9410 - val_loss: 1.1754 - val_accuracy: 0.7594
Epoch 3/100
98/98 [==============================] - 22s 222ms/step - loss: 0.1846 - accuracy: 0.9406 - val_loss: 1.1355 - val_accuracy: 0.7590
Epoch 4/100
98/98 [==============================] - 22s 225ms/step - loss: 0.1857 - accuracy: 0.9393 - val_loss: 1.2001 - val_accuracy: 0.7550
Epoch 5/100
98/98 [==============================] - 22s 224ms/step - loss: 0.1939 - accuracy: 0.9383 - val_loss: 1.1516 - val_accuracy: 0.7538
Epoch 6/100
98/98 [==============================] - 22s 224ms/step - loss: 0.1887 - accuracy: 0.9375 - val_loss: 1.2848 - val_accuracy: 0.7492
Epoch 7/100
98/98 [==============================] - 22s 224ms/step - loss: 0.1895 - acc

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
!rm -rf logs

In [ ]:
[1]*10**10

In [ ]:
!zip -r /content/file.zip /content
from google.colab import files
files.download("/content/file.zip")

updating: content/ (stored 0%)
updating: content/.config/ (stored 0%)
updating: content/.config/.last_survey_prompt.yaml (stored 0%)
updating: content/.config/logs/ (stored 0%)
updating: content/.config/logs/2020.06.26/ (stored 0%)
updating: content/.config/logs/2020.06.26/16.26.18.185568.log (deflated 54%)
updating: content/.config/logs/2020.06.26/16.26.32.322745.log (deflated 54%)
updating: content/.config/logs/2020.06.26/16.25.38.639070.log (deflated 92%)
updating: content/.config/logs/2020.06.26/16.26.31.787344.log (deflated 54%)
updating: content/.config/logs/2020.06.26/16.26.12.354762.log (deflated 87%)
updating: content/.config/logs/2020.06.26/16.25.58.989664.log (deflated 53%)
updating: content/.config/config_sentinel (stored 0%)
updating: content/.config/.last_opt_in_prompt.yaml (stored 0%)
updating: content/.config/configurations/ (stored 0%)
updating: content/.config/configurations/config_default (deflated 15%)
updating: content/.config/.last_update_check.json (deflated 22%)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>